<img align="center" src="figures/course.png" width="800">

#                                    16720 (B) Neural Networks for Recognition - Assignment 3

     Instructor: Kris Kitani                       TAs: Arka, Jinkun, Rawal, Rohan, Sheng-Yu

## Q1 Implementing a Fully Connected Network (75 points)

**Please include all the answers to the write-up questions to HW3:PDF**. Questions are indicated either the "write-up" or "auto-grader" tag.

In [1]:
# Do Not Modify
# Do Not Import ANY other packages
import numpy as np

# use for a "no activation" layer
def linear(x):
    return x

def linear_deriv(post_act):
    return np.ones_like(post_act)

def tanh(x):
    return np.tanh(x)

def tanh_deriv(post_act):
    return 1-post_act**2

def relu(x):
    return np.maximum(x, 0)

def relu_deriv(x):
    return (x > 0).astype(np.float)

### Q1.1 Network Initialization

#### Q1.1.1 (3 points, write-up)
Why is it not a good idea to initialize a network with all zeros? If you imagine that every layer has weights and biases, what can a zero-initialized network output after training?

<font color="red">**Please include your answer to HW3:PDF**</font>

#### Q1.1.2 (3 points, auto-grader)
Implement `initialize_weights` below to initialize neural network weights with [Xavier initialization](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf), where $Var[w] = \frac{2}{n_{in}+ n_{out}} $ and $n$ is the dimensionality of the vectors. Please use an **uniform distribution** to sample random numbers (see eq 16 in [Xavier initialization](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)), we recommend using np.random.uniform().

In [2]:
def initialize_weights(in_size: int, out_size: int, params: dict, name: str='' ):
    '''
    Initialize the weights W and b for a linear layer Y = XW + b
    
    [input]
    * in_size -- the feature dimension of the input
    * out_size -- the feature dimension of the output
    * params -- a dictionary containing parameters
    * name -- name of the layer
    
    HINTS:
    (1) b should be a 1D array, not a 2D array with a singleton dimension
    '''
    W, b = None, None
    
    ## compute W and b using in_size and out_size.
    # YOUR CODE HERE

    W = np.random.uniform(-np.sqrt(6/float(in_size+out_size)), np.sqrt(6/float(in_size+out_size)), size=(in_size, out_size))
    # b = np.random.uniform(-np.sqrt(6/float(in_size+out_size)), np.sqrt(6/float(in_size+out_size)), size=(out_size))
    b = np.zeros((out_size))
    # raise NotImplementedError()

    params['W' + name] = W
    params['b' + name] = b

In [3]:
params = {}
initialize_weights(2, 25, params, 'layer1')
initialize_weights(25, 4, params, 'output')
assert(params['Wlayer1'].shape == (2, 25))
assert(params['blayer1'].shape == (25,))


#### Q1.1.3 (2 points, write-up)
Why is it a good practice to initialize the parameters using random numbers? Explain the intuition behind scaling the initializations depending on layer size (see near Fig 6 in [Xavier initialization](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf))?

<font color="red">**Please include your answer to HW3:PDF**</font>

### Q1.2 Forward Propagation

Please refer to `appendix.jpynb` for the forward propagation equations. We will be implementing the forward propagation in code here.

#### Q1.2.1 (12 points, auto-grader)
Implement `sigmoid`, along with `forward` propagation for a single layer with an activation function, namely
$y = \sigma(X W + b)$, returning the output and intermediate results for an $N \times D$ dimension input $X$, with examples along the rows, data dimensions along the columns.

In [4]:
def sigmoid(X: np.ndarray):
    '''
    A sigmoid activation function
    
    [input]
    * X -- input data [N x D]
    
    [output]
    * res -- output after the sigmoid function
    '''
    res = None
    
    ## compute res using X
    # YOUR CODE HERE
    
    res = 1/(1+np.exp(-X))
    # raise NotImplementedError()

    return res


In [5]:
test = sigmoid(np.array([-100,100]))
assert test.min() < 1e-3
assert test.max() > 1 - 1e-3


In [6]:
def forward(X: np.ndarray, params: dict, name: str='',
            activation: callable=sigmoid):
    """
    Do a forward pass

    [input]
    * X -- input data [N x D]
    * params -- a dictionary containing parameters
    * name -- name of the layer
    * activation -- the activation function (default is sigmoid)
    
    [output]
    * post_act -- output after a linear layer and activation
    """
    pre_act, post_act = None, None
    # get the layer parameters
    W = params['W' + name]
    b = params['b' + name]
    
    ## compute pre_act using X, W and b.
    ## compute post_act using pre_act.
    # YOUR CODE HERE
    
    pre_act = np.dot(X,W) + b
    post_act = activation(pre_act)
#     raise NotImplementedError()

    # store the pre-activation and post-activation values
    # these will be important in backprop
    params['cache_' + name] = (X, pre_act, post_act)

    return post_act

In [7]:
params = {'Wlayer1': np.random.rand(10, 25), 'blayer1': np.random.rand(25,)}
X = np.random.rand(3, 10)
y = forward(X, params, 'layer1')
assert 'cache_layer1' in params


#### Q1.2.2 (5 points, auto-grader)
Implement the `softmax` function. Please implement a numerically stable computation of softmax using Theory:Q2. Hint, translate the input using the maximum element.

In [8]:
def softmax(X: np.ndarray):
    """
    A softmax function.
    
    [input]
    * X -- input data [N x D]
    
    [output]
    * res -- values after softmax
    """
    res = None
    
    ## compute res using X
    # YOUR CODE HERE
    
    c = -np.max(X)
    res = (np.exp(X+c)).transpose()/np.sum(np.exp(X+c), axis=1)
    res = res.transpose()

    # res = np.exp(X+c)/np.sum(np.exp(X+c), axis=0)
    # raise NotImplementedError()

    return res


#### Q1.2.3 (5 points, auto-grader)
Implement `compute_loss_and_acc` to compute the accuracy of a set of labels, along with the scalar loss across the data.  The loss function generally used for classification is the cross-entropy loss.

$$L_{\textbf{f}}(\textbf{D}) = - \sum_{(\textbf{x}, \textbf{y})\in \textbf{D}}\textbf{y}\cdot\log(\textbf{f}(\textbf{x}))$$
Here $\textbf{D}$ is the full training dataset of data samples $\textbf{x}$ ($N\times 1$ vectors, N = dimensionality of data) and labels $\textbf{y}$ ($C\times 1$ one-hot vectors, C = number of classes).

In [9]:
def compute_loss_and_acc(y: np.ndarray, probs: np.ndarray):
    """
    Compute total loss and accuracy
    
    [input]
    * y -- one hot labels [N x C]
    * probs -- class probabities [N x C]
    
    [output]
    * loss -- cross-entropy loss
    * acc -- accuracy
    """
    loss, acc = None, None
    
    ## compute loss as a function of probs and y
    ## compute acc using probs and y
    # YOUR CODE HERE
    loss = -np.sum(y * np.log(probs))
    
    T_label = np.argmax(y, axis = 1)
    P_label = np.argmax(probs, axis = 1)
    count = 0
    for i in range(len(T_label)):
        if T_label[i] == P_label[i]:
            count += 1
    
    acc = count/y.shape[0]
    # raise NotImplementedError()

    return loss, acc

### Q1.3 Backwards Propagation

#### Q1.3.1 (10 points, auto-grader)
Compute back-propagation for a single layer, given the original weights, the appropriate intermediate results, and given gradient with respect to the loss. You should return the gradient with respect to $X$ so you can feed it into the next layer. As a sanity check, your gradients should be the same dimensions as the original objects.

In [10]:
def sigmoid_deriv(post_act: np.ndarray):
    """
    Derivative of sigmoid.
    
    we give this to you because you proved it
    it's a function of post_act
    """
    res = post_act*(1.0-post_act)
    return res


def backwards(delta: np.ndarray, params: dict, name: str='',
              activation_deriv: callable=sigmoid_deriv):
    """
    Do a backwards pass

    [input]
    * delta -- errors to backprop
    * params -- a dictionary containing parameters
    * name -- name of the layer
    * activation_deriv -- the derivative of the activation_func
    
    [output]
    * grad_X -- gradient w.r.t X
    """
    grad_X, grad_W, grad_b = None, None, None
    # everything you may need for this layer
    W = params['W' + name]
    b = params['b' + name]
    
    X, pre_act, post_act = params['cache_' + name]
    # print(X.shape)
    # do the derivative through activation first
    # then compute the derivative W,b, and X
    # YOUR CODE HERE
    # print('pre actka shape = ', pre_act.shape)
    # print('X ka shape = ', X.shape)
    # print(activation_deriv(post_act).shape)
    # print(grad_b.shape)
    # print(W.shape)
    # print(delta.shape)
    # print(grad_W.shape)
    # grad_X = activation_deriv(post_act)*W*delta
    # grad_b = np.dot(activation_deriv(post_act), delta.transpose())
    # grad_W = activation_deriv(post_act)*X*delta
    
    # act_dash = activation_deriv(post_act)
    act_dash = activation_deriv(post_act)
    # print(np.ones((act_dash.shape[0])))
    grad_b = np.dot(np.ones((act_dash.shape[0])), (act_dash*delta))
    grad_W = np.dot(X.transpose(), (delta * act_dash))
    grad_X = np.dot((act_dash*delta), W.transpose())
    
    # # raise NotImplementedError()

    # store the gradients
    params['grad_W' + name] = grad_W
    params['grad_b' + name] = grad_b
    # np.save('kshitij_W.npy', grad_W)
    # np.save('kshitij_X.npy', grad_X)
    # np.save('kshitij_b.npy', grad_b)
    return grad_X

In [11]:
# we use random values to test your implementation 
# independent of previous questions
n, c1, c2 = 5, 40, 20 
rng = np.random.RandomState(12345)
delta = np.random.rand(n, c2)
name = 'layer1'
params = {
    'W'+name: np.random.rand(c1, c2),
    'b'+name: np.random.rand(c2),
    'cache_'+name: (np.random.rand(n, c1), 
                     np.random.rand(n, c2), 
                     np.random.rand(n, c2))
}
print()
grad = backwards(delta, params, name, tanh_deriv)

assert 'grad_W' + name in params
assert 'grad_b' + name in params

assert params['grad_W'+name].shape == params['W'+name].shape
assert params['grad_b'+name].shape == params['b'+name].shape


### Q1.4 Convolutional Layer (10 points)

For now we have worked with linear layer in fully-connected networks. In practice, convolutional layers are commonly used to extract image feature. You will implement the forward and backawad propagation for convolutional layer in this subsection. 

#### Q1.4.1 (5 points, auto-grader)
Similar to Q1.2.1, implement `conv_forward` for a single convolutional layer with zero paddings.

In [12]:
a = np.array([[[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]]])
b,c = a.shape[0],a.shape[1]
print(b,c)

1 3


In [13]:
def conv_forward(X: np.ndarray, params: dict, name: str='',
            stride: int=1, pad: int=0):
    """
    Do a forward pass for a convolutional layer

    [input]
    * X -- input data [N x C x H x W]
    * params -- a dictionary containing parameters
    * name -- name of the layer
    * stride, pad -- convolution parameters
    
    [output]
    * res -- output after a convolutional layer
    """
    res = None
    # get the layer parameters
    w = params['W' + name] # Conv Filter weights [F x C x HH x WW]
    b = params['b' + name] # Biases [F]
    
    # YOUR CODE HERE
    row, col = X.shape[2], X.shape[3]
    
    h_row, h_col = w.shape[2], w.shape[3]
    height = 1+int(((row-h_row+(2*pad)))/stride)
    width = 1+int(((col-h_col+(2*pad)))/stride)
#     res = np.zeros((N,F,H,W))
    res = np.zeros((X.shape[0],w.shape[0],height,width))
    # print('res shape = ', res.shape)
    
    for img_no in range(X.shape[0]):
        windows = []
        curr = X[img_no,:,:,:]
        X_pad = np.pad(curr, ((0,0),(pad,pad),(pad,pad)), mode='constant')
        for i in range(height):
                for j in range(width):
                        window = X_pad[:,i*stride:(i*stride)+h_row, j*stride:(j*stride)+h_col]
                        # print('window shape = ', window.shape)
                        windows.append(window.reshape(-1, 1))

        image_to_convolve = np.concatenate(windows, axis=1)
        # print('img2conv shape = ',image_to_convolve.shape)
        
        for filt_no in range(w.shape[0]):
                filt = w[filt_no,:,:,:]
                filt = filt.reshape(-1,1)
                # print('filt shape before tile = ', filt.shape)
                filt_window = np.tile(filt, (1,image_to_convolve.shape[-1]))
                # print('filt shape after tile = ', filt_window.shape)
                curr_conv = image_to_convolve * filt_window
                curr_conv = np.sum(curr_conv, axis=0)
                curr_conv = np.reshape(curr_conv, (height, width))
                curr_conv = curr_conv + b[filt_no]
                res[img_no,filt_no,:,:] = curr_conv
#     raise NotImplementedError()

    # store the input and convolution parameters
    # these will be important in backprop
    params['cache_' + name] = (X, stride, pad)

    return res

In [14]:
x_shape = np.array((2, 3, 4, 4))
w_shape = np.array((3, 3, 4, 4))
x = np.linspace(-0.1, 0.5, num=np.prod(x_shape), dtype=np.float64).reshape(*x_shape)
w = np.linspace(-0.2, 0.3, num=np.prod(w_shape), dtype=np.float64).reshape(*w_shape)
b = np.linspace(-0.1, 0.2, num=3, dtype=np.float64)

params = {'WConv_layer1': w, 'bConv_layer1': b}
y = conv_forward(np.array(x), params, 'Conv_layer1', stride=2, pad=1)
assert 'cache_Conv_layer1' in params


y_ref = np.array([[[[-0.08759809, -0.10987781],
                              [-0.18387192, -0.2109216 ]],
                             [[ 0.21027089,  0.21661097],
                              [ 0.22847626,  0.23004637]],
                             [[ 0.50813986,  0.54309974],
                              [ 0.64082444,  0.67101435]]],
                            [[[-0.98053589, -1.03143541],
                              [-1.19128892, -1.24695841]],
                             [[ 0.69108355,  0.66880383],
                              [ 0.59480972,  0.56776003]],
                             [[ 2.36270298,  2.36904306],
                              [ 2.38090835,  2.38247847]]]], 
            )
# print('y shape = ', y.shape)
# print('y_ref shape = ', y_ref.shape)
assert y.shape == y_ref.shape

max_diff = np.max(np.abs((y_ref - y)))
base = (np.abs(y_ref) + np.abs(y)).clip(np.finfo(float).eps).max()
print(max_diff/base) # the difference should be less than 1e-8


1.0141824738238694e-09


#### Q1.4.2 (5 points, auto-grader)
Implement `conv_backword` for a single convolutional layer with zero paddings.
Compute back-propagation for a single convolutional layer, given the original weights, the cached input, and given gradient with respect to the loss. Similar to Q1.3.1, you should return the gradient with respect to $X$ so you can feed it into the next layer. As a sanity check, your gradients should be the same dimensions as the original objects.

In [15]:
def conv_backward(delta: np.ndarray, params: dict, name: str=''):
    """
    Do a backwards pass for a convolutional layer

    [input]
    * delta -- errors to backprop
    * params -- a dictionary containing parameters
    * name -- name of the layer
    
    [output]
    * grad_X -- gradient w.r.t X
    """
    grad_X, grad_W, grad_b = None, None, None
    # everything you may need for this layer
    W = params['W' + name]
    b = params['b' + name]
    X, stride, pad = params['cache_' + name]

    # print('X shape = ', X.shape)
    # print('W shape = ', W.shape)
    # print('b shape = ', b.shape)
    # print('delta shape = ', delta.shape)
    # X shape =  (5, 4, 16, 16)
    # delta shape =  (5, 8, 8, 8)
    # W shape =  (8, 4, 7, 7)
    # b shape =  (8,)
    
    # compute the derivative W,b, and X
    # YOUR CODE HERE

    row, col = X.shape[2], X.shape[3]
    h_row, h_col = W.shape[2], W.shape[3]
    height = 1+int(((row-h_row+(2*pad)))/stride)
    width = 1+int(((col-h_col+(2*pad)))/stride)
    # print('height  = ', height)
#     res = np.zeros((N,F,H,W))
    grad_W = np.zeros(W.shape)
    # print('grad_W ka shape = ', grad_W.shape)
    # print('grad_W shape = ', grad_W.shape)
    
    ##################### Good #########################
    # for img_no in range(X.shape[0]):
    #     # windows = np.zeros((delta.shape[2], delta.shape[3]))
    #     windows = []
    #     curr = X[img_no,:,:,:]
    #     X_pad = np.pad(curr, ((0,0),(pad,pad),(pad,pad)), mode='constant')
    #     for filt_no in range(delta.shape[1]):
    #         for i in range(height):
    #                 for j in range(width):
    #                         window = X_pad[:,i*stride:(i*stride)+h_row, j*stride:(j*stride)+h_col]
    #                         temp = window * delta[img_no,filt_no,i,j]
    #                         # print('temp shape = ', temp.shape)
    #                         grad_W[filt_no,:,:,:] += temp
    ##################### Good #########################
    
    # grad_W1 = np.zeros(W.shape)
    ##################### Better #######################
    for img_no in range(X.shape[0]):
        # windows = np.zeros((delta.shape[2], delta.shape[3]))
        windows = []
        curr = X[img_no,:,:,:]
        X_pad = np.pad(curr, ((0,0),(pad,pad),(pad,pad)), mode='constant')
        # for filt_no in range(delta.shape[1]):
        for i in range(height):
                for j in range(width):
                        window = X_pad[:,i*stride:(i*stride)+h_row, j*stride:(j*stride)+h_col]
                        # temp = window * delta[img_no,filt_no,i,j]
                        # print('temp shape = ', temp.shape)
                        # grad_W[filt_no,:,:,:] += temp
                            
                        # print('window shape = ', window.shape)
                        windows.append(window.reshape(-1, 1))
                        
                        # windows.append(window)
                        # print('windows ka size = ', len(windows))
        # windows = np.array(windows)
        # print('windows array shape = ', windows.shape)
        # image_to_convolve = np.concatenate(windows, axis=1)
        # print('img2conv shape = ',image_to_convolve.shape)
        
        for filt_no in range(delta.shape[1]):
                filt = delta[img_no,filt_no,:,:]
                filt = filt.reshape(-1,1)
                # print('filt shape before tile = ', filt.shape)
                filt_window = np.tile(filt, (1,window.shape[0]*window.shape[1]*window.shape[2]))
                # filt_window = np.tile(filt, 10)
                # print('filt shape after tile = ', filt_window.shape)
                filt_window = filt_window[:,:,np.newaxis]
                curr_conv = np.multiply(windows, filt_window)
                # print('curr conv shape 1 = ', curr_conv.shape) 
                curr_conv = np.sum(curr_conv, axis=0)
                # print('curr conv shape 2 = ', curr_conv.shape)
                curr_conv = np.reshape(curr_conv, (W.shape[1],height-1, width-1))
                # curr_conv = curr_conv + b[filt_no]
                grad_W[filt_no,:,:,:] += curr_conv
        # print('grad_W shape = ', grad_W.shape)
    ##################### Better #######################

    grad_X = np.zeros(X.shape)
    grad_X_pad = np.pad(grad_X, ((0,0),(0,0),(pad,pad),(pad,pad)))
    for N in range(delta.shape[0]):
        del_layer = delta[N,:,:,:]
        for i, j in zip(range(delta.shape[1]), range(W.shape[0])):
            for p in range(delta.shape[2]):
                for q in range(delta.shape[3]):
                    X_window =  W[j,:,:,:] * delta[N,i,p,q]
                    # print('X window shape = ', X_window.shape)
                    grad_X_pad[N,:, p*stride:(p*stride)+W.shape[2], q*stride:(q*stride)+W.shape[3]] += X_window
    grad_X = grad_X_pad[:,:,pad:grad_X_pad.shape[2]-pad,pad:grad_X_pad.shape[3]-pad]

    grad_b = np.sum(delta, axis=(0,2,3))
    # raise NotImplementedError()
    # print('difference = ', np.sum(grad_W - grad_W1))
    # store the gradients
    params['grad_W' + name] = grad_W
    params['grad_b' + name] = grad_b
    return grad_X

In [16]:
x = np.random.rand(5, 4, 16, 16)
w = np.random.rand(8, 4, 7, 7)
b = np.random.rand(8,)
dout = np.random.rand(5, 8, 8, 8)

params = {'WConv_layer1': w, 'bConv_layer1': b}
y = conv_forward(x, params, 'Conv_layer1', stride=2, pad=3)
dx = conv_backward(dout, params, 'Conv_layer1')
assert x.shape == dx.shape
assert params['grad_WConv_layer1'].shape == params['WConv_layer1'].shape
assert params['grad_bConv_layer1'].shape == params['bConv_layer1'].shape


### Q1.5 The Training Loop
You usually see gradient descent in three forms: "normal", "stochastic" and "batch". "Normal" gradient descent aggregates the updates for the entire dataset before changing the weights. Stochastic gradient descent applies updates after every single data example. Batch gradient descent is a compromise, where random subsets of the full dataset are evaluated before applying the gradient update. 

#### Q1.5.1 (10 points, auto-grader)
Write a training loop that generates random batches, iterates over them for many iterations, does forward and backward propagation, and applies a gradient update step. Specifically, implement `get_random_batches` and `train` functions below.

In [19]:
def get_random_batches(x: np.ndarray, y: np.ndarray, batch_size: int) -> list:
    import math
    """
    Split x and y into random batches
    
    [input]
    * x -- training samples
    * y -- training lables
    * batch_size -- batch size
    
    [output]
    * batches -- a list of [(batch1_x,batch1_y)...]
    """
    # 
    # return 
    batches = []
    # YOUR CODE HERE
    random_x = np.zeros(x.shape)
    random_y = np.zeros(y.shape)
    shuffeled = np.random.permutation(len(x))
    # for old_idx, new_idx in enumerate(shuffeled):
    #     random_x[new_idx] = x[old_idx]
    #     random_y[new_idx] = y[old_idx]
    random_x = x[shuffeled]
    random_y = y[shuffeled]
    
    no_of_batches = math.ceil(x.shape[0]/batch_size)
    for batchno in range(no_of_batches):
        batches.append((random_x[batchno*batch_size:(batchno+1)*batch_size], random_y[batchno*batch_size:(batchno+1)*batch_size]))

    # raise NotImplementedError()
    return batches


In [20]:
n, c1, c2 = 20, 100, 5
# n, c1, c2 = 2, 2, 2
batch_size = 3
x = np.random.rand(n, c1)
y = np.random.rand(n, c2)
batches = get_random_batches(x, y, batch_size)
assert type(batches) == list
assert len(batches) >= 6


In [21]:
def train(x: np.ndarray, y: np.ndarray, params: dict, batch_size: int = 5,
          max_iters: int = 500, learning_rate: float=1e-3):
    
    """
    Train the network with two sequential layers: 
    (1) one layer named "layer1" with sigmoid activation
    (2) one layer named "output" with softmax activation

    [input]
    * x -- training samples
    * y -- training lables
    * params -- a dictionary containing initial parameters
    * batch_size -- batch size
    * max_iters -- total number of iterations
    * learning_rate -- learning rate
    
    [output]
    * total_loss, avg_acc -- loss and accuracy for the last iteration
    """

    batches = get_random_batches(x, y, batch_size)

    for itr in range(max_iters):
        total_loss = 0
        avg_acc = 0
        for xb, yb in batches:

            # forward
            # YOUR CODE HERE
            post_act = forward(xb,params,'layer1',sigmoid)
            pred_output = forward(post_act,params,'output',softmax)
            # raise NotImplementedError()
            
            # loss
            # be sure to add loss and accuracy to epoch totals
            # YOUR CODE HERE
            loss, acc = compute_loss_and_acc(yb, pred_output)
            total_loss += loss
            avg_acc += acc/len(batches)
            # raise NotImplementedError()
            
            # backward
            # YOUR CODE HERE
            last_layer_backprop = backwards(pred_output - yb, params, 'output', linear_deriv)
            hidden_layer_backprop  = backwards(last_layer_backprop, params, 'layer1', sigmoid_deriv)
            # raise NotImplementedError()

            # apply gradient
            # YOUR CODE HERE
            params['Woutput'] = params['Woutput'] - learning_rate*params['grad_Woutput']
            params['boutput'] = params['boutput'] - learning_rate*params['grad_boutput']
            params['Wlayer1'] = params['Wlayer1'] - learning_rate*params['grad_Wlayer1']
            params['blayer1'] = params['blayer1'] - learning_rate*params['grad_blayer1']
            # raise NotImplementedError()
            
        if itr % 100 == 0:
            print("itr: {:02d} \t loss: {:.2f} \t acc : {:.2f}".format(
                itr, total_loss, avg_acc))
    return total_loss, avg_acc


In [22]:
# Successulf implementation of dependent functions are required to get full score for the `train` function

# create inputs
g0 = np.random.multivariate_normal([3.6,40],[[0.05,0],[0,10]],10)
g1 = np.random.multivariate_normal([3.9,10],[[0.01,0],[0,5]],10)
g2 = np.random.multivariate_normal([3.4,30],[[0.25,0],[0,5]],10)
g3 = np.random.multivariate_normal([2.0,10],[[0.5,0],[0,10]],10)
x = np.vstack([g0,g1,g2,g3])

# create labels
y_idx = np.array([0 for _ in range(10)] + [1 for _ in range(10)] + [2 for _ in range(10)] + [3 for _ in range(10)])

# turn labels to one_hot
y = np.zeros((y_idx.shape[0],y_idx.max()+1))
y[np.arange(y_idx.shape[0]),y_idx] = 1

# parameters in a dictionary
params = {}
# initialize a layer
initialize_weights(2,25,params,'layer1')
initialize_weights(25,4,params,'output')

# train the two-layer neural network
total_loss, avg_acc = train(x, y, params, batch_size=5, max_iters=500, learning_rate=1e-3)
print("itr: {:02d} \t loss: {:.2f} \t acc : {:.2f}".format(500, total_loss, avg_acc))

# with default settings, you should get loss < 35 and accuracy > 70%
assert total_loss < 35 and avg_acc > 0.70


itr: 00 	 loss: 59.82 	 acc : 0.25
itr: 100 	 loss: 42.56 	 acc : 0.55
itr: 200 	 loss: 35.67 	 acc : 0.62
itr: 300 	 loss: 31.87 	 acc : 0.70
itr: 400 	 loss: 29.75 	 acc : 0.72
itr: 500 	 loss: 28.30 	 acc : 0.77


### Q1.6 Numerical Gradient Checker

#### Q1.6.1 (15 points, auto-grader)
Implement the `centeral_differences_gradient` function. Instead of using the analytical gradients computed from the chain rule, add $\epsilon$ offset to each element in the weights, and compute the numerical gradient of the loss with central differences. Central differences is just $\frac{f(x+\epsilon) - f(x-\epsilon)}{2 \epsilon}$. Remember, this needs to be done for each scalar dimension in all of your weights independently. 

In [23]:
def centeral_differences_gradient(params: dict, eps = 1e-6):
    """
    Compute the estimated gradients using central difference
    
    Hint:
    please feel free to reuse the functions above
    """
    for k, v in params.items():
        if '_' in k:
            continue
        # we have a real parameter!
        # for each value inside the parameter
        #   subtract epsilon
        #   run the forward function by 
        #   get the loss
        #   add epsilon
        #   run the forward function
        #   get the loss
        #   compute derivative with central diffs
    
        # YOUR CODE HERE
        if k == 'Woutput' or k == 'Wlayer1':
            for i in range(v.shape[0]):
                for j in range(v.shape[1]):
                    v[i][j] = v[i][j] - eps
                    fwd_prop11 = forward(x,params,'layer1')
                    op1 = forward(fwd_prop11, params, 'output', softmax)
                    loss1, acc1 = compute_loss_and_acc(y, op1)
                    v[i][j] = v[i][j] + 2*eps
                    fwd_prop21 = forward(x,params,'layer1')
                    op2 = forward(fwd_prop21, params, 'output', softmax)
                    loss2, acc2 = compute_loss_and_acc(y, op2)
                    params["grad_"+k][i][j] = (loss2 - loss1)/(2*eps)
                    v[i][j] = v[i][j] - eps
        elif k == 'boutput' or k == 'blayer1':
            for i in range(v.shape[0]):
                v[i] = v[i] - eps
                fwd_prop11 = forward(x,params,'layer1')
                op1 = forward(fwd_prop11, params, 'output', softmax)
                loss1, acc1 = compute_loss_and_acc(y, op1)
                v[i] = v[i] + 2*eps
                fwd_prop21 = forward(x,params,'layer1')
                op2 = forward(fwd_prop21, params, 'output', softmax)
                loss2, acc2 = compute_loss_and_acc(y, op2)
                params['grad_'+k][i] = (loss2 - loss1)/(2*eps)                
                v[i] = v[i] - eps
        # raise NotImplementedError()


In [24]:
# Compute the analytical gradients
h1 = forward(x,params,'layer1')
probs = forward(h1,params,'output',softmax)
delta1 = probs
delta1[np.arange(probs.shape[0]),y_idx] -= 1

delta2 = backwards(delta1,params,'output',linear_deriv)
backwards(delta2,params,'layer1',sigmoid_deriv)

import copy
params_orig = copy.deepcopy(params)

# Compute the estimated gradient using central difference
centeral_differences_gradient(params)

total_error = 0
for k in params.keys():
    if 'grad_' in k:
        # relative error
        err = np.abs(params[k] - params_orig[k])/np.maximum(np.abs(params[k]),np.abs(params_orig[k]))
        err = err.sum()
        total_error += err
# should be less than 1e-4
print('erroe = ', total_error)
assert 0. < total_error < 1e-4
print('Test case passed, good job!')

erroe =  7.766246036131394e-06
Test case passed, good job!
